# Homework 6: Web Applications

For this homework, you're going to write a web API for the lake data in the MONDIAL database. (Make sure you've imported the data as originally outlined in [our week 1 tutorial](https://github.com/ledeprogram/data-and-databases/blob/master/SQL_notes.md).)

The API should perform the following tasks:

* A request to `/lakes` should return a JSON list of dictionaries, with the information from the `name`, `elevation`, `area` and `type` fields from the `lake` table in MONDIAL.
* The API should recognize the query string parameter `sort`. When left blank or set to `name`, the results should be sorted by the name of the lake (in alphabetical order). When set to `area` or `elevation`, the results should be sorted by the requested field, in *descending* order.
* The API should recognize the query string parameter `type`. When specified, the results should *only* include rows that have the specified value in the `type` field.
* You should be able to use *both* the `sort` and `type` parameters in any request.

This notebook contains only *test requests to your API*. Write the API as a standalone Python program, start the program and then run the code in the cells below to ensure that your API produces the expected output. When you're done, paste the source code in the final cell (so we can check your work, if needed).

Hints when writing your API code:

* You'll need to construct the SQL query as a string, piece by piece. This will likely involve a somewhat messy tangle of `if` statements. Lean into the messy tangle.
* Make sure to use parameter placeholders (%s) in the query.
* If you're getting SQL errors, print out your SQL statement in the request handler function so you can debug it. (When you use `print()` in Flask, the results will display in your terminal window.)
* When in doubt, return to the test code. Examine it carefully and make sure you know exactly what it's trying to do.

## Problem set #1: A list of lakes

Your API should return a JSON list of dictionaries (objects). Use the code below to determine what the keys of the dictionaries should be. (For brevity, this example only prints out the first ten records, but of course your API should return all of them.)

Expected output:

    143 lakes
    Ammersee - elevation: 533 m / area: 46 km^2 / type: None
    Arresoe - elevation: None m / area: 40 km^2 / type: None
    Atlin Lake - elevation: 668 m / area: 798 km^2 / type: None
    Balaton - elevation: 104 m / area: 594 km^2 / type: None
    Barrage de Mbakaou - elevation: None m / area: None km^2 / type: dam
    Bodensee - elevation: 395 m / area: 538 km^2 / type: None
    Brienzersee - elevation: 564 m / area: 29 km^2 / type: None
    Caspian Sea - elevation: -28 m / area: 386400 km^2 / type: salt
    Chad Lake - elevation: 250 m / area: 23000 km^2 / type: salt
    Chew Bahir - elevation: 520 m / area: 800 km^2 / type: salt

In [ ]:
import requests
data = requests.get('http://localhost:5000/lakes').json()
print(len(data), "lakes")
for item in data[:10]:
    print(item['name'], "- elevation:", item['elevation'], "m / area:", item['area'], "km^2 / type:", item['type'])

## Problem set #2: Lakes of a certain type

The following code fetches all lakes of type `salt` and finds their average area and elevation.

Expected output:

    average area: 18880
    average elevation: 970

In [ ]:
import requests
data = requests.get('http://localhost:5000/lakes?type=salt').json()
avg_area = sum([x['area'] for x in data if x['area'] is not None]) / len(data)
avg_elev = sum([x['elevation'] for x in data if x['elevation'] is not None]) / len(data)
print("average area:", int(avg_area))
print("average elevation:", int(avg_elev))

## Problem set #3: Lakes in order

The following code fetches lakes in reverse order by their elevation and prints out the name of the first fifteen, excluding lakes with an empty elevation field.

Expected output:

    * Licancabur Crater Lake
    * Nam Co
    * Lago Junin
    * Lake Titicaca
    * Poopo
    * Salar de Uyuni
    * Koli Sarez
    * Lake Irazu
    * Qinghai Lake
    * Segara Anak
    * Lake Tahoe
    * Crater Lake
    * Lake Tana
    * Lake Van
    * Issyk-Kul

In [ ]:
import requests
data = requests.get('http://localhost:5000/lakes?sort=elevation').json()
for item in [x['name'] for x in data if x['elevation'] is not None][:15]:
    print("*", item)

## Problem set #4: Order and type

The following code prints the names of the largest caldera lakes, ordered in reverse order by area.

Expected output:

    * Lake Nyos
    * Lake Toba
    * Lago Trasimeno
    * Lago di Bolsena
    * Lago di Bracciano
    * Crater Lake
    * Segara Anak
    * Laacher Maar

In [ ]:
import requests
data = requests.get('http://localhost:5000/lakes?sort=area&type=caldera').json()
for item in data:
    print("*", item['name'])

## Problem set #5: Error handling

Your API should work fine even when faced with potential error-causing inputs. For example, the expected output for this statement is an empty list (`[]`), *not* every row in the table.

In [43]:
import requests
data = requests.get('http://localhost:5000/lakes', params={'type': "' OR true; --"}).json()
data

[]

Specifying a field other than `name`, `area` or `elevation` for the `sort` parameter should fail silently, defaulting to sorting alphabetically. Expected output: `['Ammersee', 'Arresoe', 'Atlin Lake', 'Balaton', 'Barrage de Mbakaou']
`

In [46]:
import requests
data = requests.get('http://localhost:5000/lakes', params={'sort': "florb"}).json()
[x['name'] for x in data[:5]]

['Ammersee', 'Arresoe', 'Atlin Lake', 'Balaton', 'Barrage de Mbakaou']

## Paste your code

Please paste the code for your entire Flask application in the cell below, in case we want to take a look when grading or debugging your assignment.

In [58]:
conn.rollback()

In [ ]:


from flask import Flask, request, jsonify
from decimal import Decimal
import pg8000

app = Flask(__name__)
conn = pg8000.connect(user="postgres",password="password", database="mondial")
@app.route("/lakes")

def get_lakes():
    sorting = request.args.get('sort', 'name')       
    get_type = request.args.get('type', 0)
    
    cursor = conn.cursor()
    
    sort_by = "ORDER BY name"
    if sorting=='area':
        sort_by = "ORDER BY area DESC"
    elif sorting=='elevation':
        sort_by = "ORDER BY elevation DESC"
    elif sorting=='name':
        sort_by = "ORDER BY name DESC"
 
    if get_type:
        cursor.execute("SELECT name, area, elevation, type FROM lake WHERE type=%s " + sort_by, [get_type])
    else:
        cursor.execute("SELECT name, area, elevation, type FROM lake " + sort_by)
     
    def decimal_to_int(x):
        if isinstance(x, decimal.Decimal):
            return int(x)
        else:
            return None
    
    output = []
    for item in cursor.fetchall():
        try:
            dictionary = {'name':item[0],'area': int(item[1]),'elevation':float(item[2]), 'type':item[3]}
            print(dictionary)
            output.append(
                dictionary
            )
            
        except:
            pass
    return jsonify(output)
                           
app.run(port=5004)

{'type': 'salt', 'area': 10582, 'name': 'Salar de Uyuni', 'elevation': 3650.0}
{'type': 'salt', 'area': 4583, 'name': 'Qinghai Lake', 'elevation': 3195.0}
{'type': 'salt', 'area': 487, 'name': 'Pyramid Lake', 'elevation': 1155.0}
{'type': 'salt', 'area': 1340, 'name': 'Poopo', 'elevation': 3686.0}
{'type': 'salt', 'area': 18428, 'name': 'Ozero Balchash', 'elevation': 342.0}
{'type': 'salt', 'area': 17160, 'name': 'Ozero Aral', 'elevation': 31.0}
{'type': 'salt', 'area': 1855, 'name': 'Nam Co', 'elevation': 4718.0}
{'type': 'salt', 'area': 5000, 'name': 'Lop Nor', 'elevation': 780.0}
{'type': 'salt', 'area': 3740, 'name': 'Lake Van', 'elevation': 1719.0}
{'type': 'salt', 'area': 5470, 'name': 'Lake Urmia', 'elevation': 1280.0}
{'type': 'salt', 'area': 6405, 'name': 'Lake Turkana', 'elevation': 375.0}
{'type': 'salt', 'area': 5760, 'name': 'Lake Rukwa', 'elevation': 800.0}
{'type': 'salt', 'area': 1040, 'name': 'Lake Natron', 'elevation': 600.0}
{'type': 'salt', 'area': 9500, 'name': 'La

##### The API should recognize the query string parameter type. When specified, the results should only include rows that have the specified value in the type field.

In [36]:
conn.rollback()